In [ ]:
#%pip install kagglehub

Note: you may need to restart the kernel to use updated packages.Collecting kagglehub




[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import plotly.express as px
import plotly.graph_objects as go
import kagglehub
from kagglehub import KaggleDatasetAdapter

## 1. Data

In [ ]:
dolarbo = pd.read_excel('./DATA/dolarbo_time_series.xlsx', index_col=0)
dolarbo.index = pd.to_datetime(dolarbo.index)

In [77]:
usdt_luis = pd.read_excel('./DATA/usdt_lucho.xlsx')
# Ensure column names are stripped of extra spaces
usdt_luis.columns = usdt_luis.columns.str.strip()
# Combine 'Fecha' and 'hora' columns to create a 'timestamp' column
usdt_luis['timestamp'] = pd.to_datetime(usdt_luis['Fecha'].astype(str) + " " + usdt_luis['hora'].astype(str))
usdt_luis.index = usdt_luis['timestamp']
usdt_luis = usdt_luis.drop(['Fecha', 'hora', 'timestamp'], axis=1)
usdt_luis

,Compra,Venta
timestamp,,
2023-09-21 18:00:00,7.40,7.55
2023-09-22 18:00:00,7.40,7.55
2023-11-18 10:00:00,7.41,7.56
2023-12-12 17:00:00,7.55,7.65
2024-02-22 09:47:00,8.12,8.40
...,...,...
2025-03-11 16:56:00,11.10,11.60
2025-03-12 12:34:00,11.20,11.70
2025-03-12 18:26:00,11.50,12.00


In [98]:
usdtbol = pd.read_excel('./DATA/usdtbol_time_series.xlsx')
usdtbol['timestamp'] = pd.to_datetime(usdtbol['category'], format='%a %b %d %Y')
usdtbol.index = usdtbol['timestamp']
usdtbol = usdtbol.drop(['category', 'timestamp'], axis=1)
#usdtbol['aux'] = 1
#usdtbol['within_day'] = usdtbol.resample('D')['aux'].transform('sum')
usdtbol

,Bs
timestamp,
2024-08-06,12.89
2024-08-06,12.86
2024-08-06,12.82
2024-08-06,12.72
2024-08-06,12.44
...,...
2025-04-14,13.43
2025-04-14,13.42
2025-04-14,13.41


In [102]:
usdtbol_day = usdtbol.resample('D').mean()

In [104]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=dolarbo.index, y=dolarbo['compra'], mode='lines', name='Buy', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=dolarbo.index, y=dolarbo['venta'], mode='lines', name='Sell', line=dict(color='red')))
#fig.add_trace(go.Scatter(x=usdt_luis.index, y=usdt_luis['Compra'], mode='lines', name='Sell', line=dict(color='cyan')))
#fig.add_trace(go.Scatter(x=usdt_luis.index, y=usdt_luis['Venta'], mode='lines', name='Sell', line=dict(color='green')))
fig.add_trace(go.Scatter(x=usdtbol_day.index, y=usdtbol_day['Bs'], mode='lines', name='usdtbol.com', line=dict(color='green')))
fig.update_layout(
    title='BOB/USDT',
    xaxis_title='Timestamp',
    yaxis_title='Price',
    legend_title='Type',
    template='plotly_white'
)

fig.show()

## 2. Ultra-High Frequency Data

History Exchange Rate USDT/BOB with hourly frequency [(More info)](https://www.kaggle.com/datasets/darlynbravo/history-exchange-rate-usdtbob).

- **price:** exchange rate
- **available:** amount available
- **advertisers_qty:** quantity of advertisers
- **type:** bid (the highest price a buyer will pay) or ask (the lowest price a seller will accept)
- **timestamp:** captured datetime
- **curr_from:** Currency from
- **curr_to:** Currency to
- **source:** source of data

|                              | Bid                                     | Ask                                      |
|------------------------------|-----------------------------------------|------------------------------------------|
| Definition                   | Price to sell an asset                  | Price to buy an asset                    |
| Relationship to market price | Always slightly lower than market price | Always slightly higher than market price |

Factors that affect bid and ask prices: 
- Market liquidity
- Trading volume
- Supply and demand of currencies
- Overall market volatility
- Economic releases
- Bank holidays   


Measuring the bid-ask spread: The smallest whole unit measurement of the bid-ask spread is called a pip. One pip equals 1/100 of 1%, or 0.0001. 

In [ ]:
#kaggle_df = pd.read_csv('./kaggle_currency_exchange_rates.csv')
kaggle_df = pd.read_excel('./DATA/kaggle_currency_exchange_rates.xlsx')
kaggle_df.index = kaggle_df['timestamp']
kaggle_df.index = pd.to_datetime(kaggle_df.index)

kaggle_bid = kaggle_df[kaggle_df['type']=="bid"]
kaggle_ask = kaggle_df[kaggle_df['type']=="ask"]

In [44]:
hourly_minute_bid = kaggle_bid['price'].copy().resample('T').mean()
hourly_minute_bid.dropna(axis=0, inplace=True)

hourly_minute_ask = kaggle_ask['price'].copy().resample('T').mean()
hourly_minute_ask.dropna(axis=0, inplace=True)

In [47]:
seconds_bid = kaggle_bid['price'].copy().resample('S').mean()
seconds_bid.dropna(axis=0, inplace=True)
seconds_ask = kaggle_ask['price'].copy().resample('S').mean()
seconds_ask.dropna(axis=0, inplace=True)

In [49]:
seconds_bid.info()

<class 'pandas.core.series.Series'>
DatetimeIndex: 634 entries, 2024-07-09 17:43:37 to 2024-09-02 13:30:03
Series name: price
Non-Null Count  Dtype  
--------------  -----  
634 non-null    float64
dtypes: float64(1)
memory usage: 9.9 KB


In [48]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=seconds_bid.index, y=seconds_bid.values, mode='lines', name='Bid Price', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=seconds_ask.index, y=seconds_ask.values, mode='lines', name='Ask Price', line=dict(color='red')))
fig.update_layout(
    title='Bid and Ask Prices Over Time',
    xaxis_title='Timestamp',
    yaxis_title='Price',
    legend_title='Type',
    template='plotly_white'
)

fig.show()

In [46]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=hourly_minute_bid.index, y=hourly_minute_bid.values, mode='lines', name='Bid Price', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=hourly_minute_ask.index, y=hourly_minute_ask.values, mode='lines', name='Ask Price', line=dict(color='red')))
fig.update_layout(
    title='Bid and Ask Prices Over Time',
    xaxis_title='Timestamp',
    yaxis_title='Price',
    legend_title='Type',
    template='plotly_white'
)

fig.show()

In [106]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=dolarbo.index, y=dolarbo['compra'], mode='lines', name='Buy', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=dolarbo.index, y=dolarbo['venta'], mode='lines', name='Sell', line=dict(color='red')))
fig.add_trace(go.Scatter(x=hourly_minute_bid.index, y=hourly_minute_bid.values, mode='lines', name='Bid Price', line=dict(color='pink')))
fig.add_trace(go.Scatter(x=hourly_minute_ask.index, y=hourly_minute_ask.values, mode='lines', name='Ask Price', line=dict(color='orange')))
fig.update_layout(
    title='BOB/USDT',
    xaxis_title='Timestamp',
    yaxis_title='Price',
    legend_title='Type',
    template='plotly_white'
)

fig.show()

In [20]:
# Resample the DataFrame
resampled_df = kaggle_df['price'].resample('M').mean()
resampled_df

timestamp
2024-07-31     9.783415
2024-08-31    10.668275
2024-09-30    10.453263
Freq: M, Name: price, dtype: float64